In [1]:
import pandas as pd
import numpy as np
import os

os.chdir('/home/kruu/git_folder/eye_tracking/')
from utils.data_processing import EyeTrackingProcessor

import torch
from torch.utils.data import DataLoader
from models.lstm_classifier import LSTMClassifier, train_classifier, evaluate_model
from utils.dataset import GazeMouseDataset

****
# Import Datasets
****

In [ ]:
data_path = "/store/kruu/eye_tracking"
files_list = os.listdir(data_path)
files_list = [os.path.join(data_path, file) for file in files_list]

tasks = ['Task 1', 'Task 2', 'Task 3', 'Task 4', 'Task 5', 'Task 6']
features = ['Recording timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y', 'Participant name']
interpolate_col = ['Recording timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']

processor = EyeTrackingProcessor()
all_data = processor.load_data(files_list)
dataset = processor.get_features(all_data, tasks, features)

#Fixed size resampling for the whole task
dataset_resampled = processor.fixed_points_resample(dataset, features, num_points=500)
dataset_resampled.fillna(0, inplace=True)

#Fixed Time step resampling
# dataset_resampled = processor.fixed_time_steps_resample(dataset, interpolate_col, timestep = 0.01)
# dataset_resampled.fillna(0, inplace=True)

/home/kruu/git_folder/eye_tracking/utils/data_processing.py:20: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t')
/home/kruu/git_folder/eye_tracking/utils/data_processing.py:20: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep='\t')


****
# Classifier
****

In [3]:
features = ['Relative timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']
model = train_classifier(dataset_resampled,
                          features,
                          batch_size=32,
                          hidden_dim=64,
                          num_layers=4,
                          learning_rate=0.001,
                          num_epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kruu (zhaw_zav) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/core/optimizer.py:378: Found unsupported keys in the optimizer configuration: {'gradient_clip_val'}

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | lstm      | LSTM             | 118 K  | train
1 | fc        | Linear           | 390    | train
2 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
118 K     Trainable params
0         Non-trainable params
118 K     Total params
0.474     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=51` in the `DataLoader` to improve performance.


/home/kruu/.conda/envs/aware/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=51` in the `DataLoader` to improve performance.


Epoch 99: 100%|██████████| 11/11 [00:20<00:00,  0.53it/s, v_num=7fw3, train_loss_step=1.440, val_loss=1.850, train_loss_epoch=1.460]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 11/11 [00:20<00:00,  0.53it/s, v_num=7fw3, train_loss_step=1.440, val_loss=1.850, train_loss_epoch=1.460]


****
# Analysis
****

In [4]:
ckpt_path = "/home/kruu/git_folder/eye_tracking/GazeMouse_Classification/2ajznad2/checkpoints/best_lstm_classifier.ckpt"
features = ['Relative timestamp', 'Gaze point X', 'Gaze point Y', 'Mouse position X', 'Mouse position Y']
    
model_load = LSTMClassifier(input_dim=5, hidden_dim=64, num_classes = 6, num_layers=4, learning_rate=0.001)
model_load.load_state_dict(torch.load(ckpt_path)["state_dict"])
model_load.eval()

LSTMClassifier(
  (lstm): LSTM(5, 64, num_layers=4, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
  (criterion): CrossEntropyLoss()
)

In [5]:
res = evaluate_model(model_load, dataset_resampled, features)

✅ Evaluation Complete: Loss = 3.0381, Accuracy = 0.1852
